In [51]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreceated sklearn

ERROR: Invalid requirement: '#'


Task1

In [52]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./blood_transfusion.csv"))
frame.describe(include='all') # 5 columns

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [53]:
# visualise, understand, story

Task2

In [102]:
preprocessed = frame.copy() 
preprocessed = (preprocessed-preprocessed.min())/(preprocessed.max()-preprocessed.min()) # Normalization
preprocessed.describe()

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,0.128469,0.092137,0.092137,0.336272,0.237968
std,0.109397,0.119170,0.119170,0.253924,0.426124
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.037162,0.020408,0.020408,0.145833,0.000000
50%,0.094595,0.061224,0.061224,0.270833,0.000000
75%,0.189189,0.122449,0.122449,0.500000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


TASK 3

In [55]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, Y_train_s, Y_test_s = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.33) # small test_size
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.66) # large test_size

Task 4

In [ ]:
# 4.1 KNN
def manhattan_distance(a, b):
    """
    This function calculates the manhattan distance between two datapoints.
    :param a: list: the first datapoint
    :param b: list: the second datapoint
    :return: float: the manhattan distance between the datapoints
    """
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 2) ** 0.5
    return distance      

def KNN_classifier(k, train_x, train_y, test_x):
    """
    This function trains the KNN classifier on the training data and predicts the class of the test data.
    :param k: int: the number of neighbors
    :param train_x: pandas.DataFrame: the train data
    :param train_y: pandas.Series: the corresponding classes of the train data
    :param test_x: pandas.DataFrame: the test data
    :return: pandas.Series: the predicted classes of the test data
    """
    test_y = pd.Series([0 for _ in range(test_x.shape[0])])
    for index, (_,testrow) in enumerate(test_x.iterrows()):
        distances = []
        for _, trainrow in train_x.iterrows():
            testvalues = list(testrow)  
            trainvalues = list(trainrow)
            distance = manhattan_distance(testvalues, trainvalues)
            distances.append(distance)
        nearest_neighbours = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
        neighbor_labels = train_y.iloc[nearest_neighbours]
        predicted_label = round(neighbor_labels.mean())
        test_y.iloc[index] = predicted_label
    return test_y

def knn_checker(predicted_y, real_y, mode="basic"):
    """
    This function checks the accuracy of the predicted classes against the real classes for the knn classifier.
    :param predicted_y: pandas.Series: predicted classes for the test data
    :param real_y: pandas.Series: real classes of the test data
    :param mode: string: way to evaluate the results
    :return: prints the results, returns nothing
    """
    #TODO switchcases
    if mode == "basic":
        num, total = 0, 0
        for index in range(len(predicted_y)):
            if predicted_y.iloc[index] == real_y.iloc[index]: 
                num += 1
            total += 1
        print(f"For KNN \nAccuracy: {num/total*100:.2f}%")
    if mode == "classification_report":
        print(classification_report(real_y, predicted_y))
    if mode == "confusion_matrix":
        confusionmatrix(pd.DataFrame(real_y), pd.DataFrame(predicted_y))
    if mode == "fbeta":
        print(fbeta_score(real_y, predicted_y, average='macro', beta=0.5))
        print(fbeta_score(real_y, predicted_y, average='weighted', beta=0.5))
        print(fbeta_score(real_y, predicted_y, average=None, beta=0.5))

knn_checker(KNN_classifier(5, X_train_s, Y_train_s, X_test_s), Y_test_s)


In [115]:
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score

#5.1 Confusion matrix
def confusionmatrix(predicted_y, real_y): # it works, im confused :)
    """
    This function calculates and prints the confusion matrix.
    :param predicted_y: pandas.DataFrame: predicted classes for the test data 
    :param real_y: pandas.DataFrame: real classes of the test data
    :return: prints the confusion matrix, returns nothing
    """
    tp, fp, tn, fn = 0, 0, 0, 0
    for index in range(len(predicted_y)):
        predicted = predicted_y.iloc[index]
        real = real_y.iloc[index]
        if predicted.item() == 1: 
            if predicted.item() == real.item(): 
                tp += 1
            else:
                fp += 1
        elif predicted.item() == 0:
            if predicted.item() == real.item(): 
                tn += 1
            else:
                fn += 1
    print(f"\t P \t\t N \n T \t {tp} \t\t {tn} \n F \t {fp} \t {fn} \n")

def accuracy_checker(X_test, y_test, pred, mode="basic"):
    """
    This function checks the accuracy of the predicted classes against the real classes.
    :param X_test: pandas.DataFrame: the test data to be classified
    :param y_test: pandas.Series: the real classes of the test data
    :param pred: predictor class: the method used to classify the test data
    :param mode: string: way to evaluate the results
    :return: prints the results, returns nothing
    """
    num, total = 0, 0
    predicted, true = [], []
    for index, row in X_test.iterrows():
        p = pred.predict(X_test.loc[[index]])
        q = list(y_test.loc[[index]])
        if p == q:
            num += 1
        total += 1
        predicted.append(p)
        true.append(q)
    
    if mode == "basic":
        print(f"Accuracy: {num/total*100:.2f}%")
    if mode == "classification_report":
        print(classification_report(true, predicted))
    if mode == "confusion_matrix":
        confusionmatrix(pd.DataFrame(true), pd.DataFrame(predicted))
    if mode == "fbeta":
        print(fbeta_score(true, predicted, average='macro', beta=0.5))
        print(fbeta_score(true, predicted, average='weighted', beta=0.5))
        print(fbeta_score(true, predicted, average=None, beta=0.5))


# 4.2 nbc
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB()
nbc.fit(X_train_s, Y_train_s)
print("For nbc:")
accuracy_checker(X_test_s, Y_test_s, nbc)

# 4.3 svc
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train_s, Y_train_s)
print("For svc:")
accuracy_checker(X_test_s, Y_test_s, svc)

# 4.4 mlp
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=250).fit(X_train_s, Y_train_s)
print("For mlp:")
accuracy_checker(X_test_s, Y_test_s, mlp)

For KNN 
Accuracy: 77.33%
For nbc:
Accuracy: 78.14%
For svc:
Accuracy: 77.33%
For mlp:
Accuracy: 77.33%


In [104]:
Y_predicted_s = KNN_classifier(5, X_train_s, Y_train_s, X_test_s)
print("Confusion matrix for nbc \n")
accuracy_checker(X_test_s, Y_test_s, nbc, mode ="confusion_matrix")
print("Confusion matrix for svc: \n")
accuracy_checker(X_test_s, Y_test_s, svc, mode ="confusion_matrix")
print("Confusion matrix for mlp: \n")
accuracy_checker(X_test_s, Y_test_s, mlp, mode ="confusion_matrix")
print("Confusion matrix for knn: \n")
knn_checker(Y_predicted_s, Y_test_s,  mode = "confusion_matrix")

accuracy_checker(X_test_s, Y_test_s, nbc, mode ="classification_report")

accuracy_checker(X_test_s, Y_test_s, svc, mode ="classification_report")

accuracy_checker(X_test_s, Y_test_s, mlp, mode ="classification_report")

knn_checker(Y_predicted_s, Y_test_s,  mode = "classification_report")

accuracy_checker(X_test_s, Y_test_s, nbc, mode ="fbeta")

accuracy_checker(X_test_s, Y_test_s, svc, mode ="fbeta")

accuracy_checker(X_test_s, Y_test_s, mlp, mode ="fbeta")

knn_checker(Y_predicted_s, Y_test_s,  mode = "fbeta")





Confusion matrix for nbc 

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
	 P 		 N 
 T 	 8 		 185 
 F 	 50 	 4 

Confusion matrix for svc: 

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
	 P 		 N 
 T 	 6 		 185 
 F 	 52 	 4 

Confusion matrix for mlp: 

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
	 P 		 N 
 T 	 5 		 186 
 F 	 53 	 3 

Confusion matrix for knn: 

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
	 P 		 N 
 T 	 19 		 172 
 F 	 39 	 17 

              precision    recall  f1-score   support

         0.0       0.79      0.98      0.87       189
         1.0       0.67      0.14      0.23        58

    accuracy                           0.78       247
   macro avg       0.73      0.56      0.55       247
weighted avg       0.76      0.78      0.72       247

              precision    recall  f1-score   support

         0.0       0.78      0.98      0.87  